In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # visualization 
import matplotlib.pyplot as plt # visualization
from collections import Counter # for count 

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Creating variable

In [ ]:
kill = pd.read_csv("../input/fatal-police-shootings-in-the-us/PoliceKillingsUS.csv",encoding="windows-1252" )
share_race_city = pd.read_csv("../input/fatal-police-shootings-in-the-us/ShareRaceByCity.csv",encoding="windows-1252" )

In [ ]:
kill.info()

In [ ]:
share_race_city.info()

In [ ]:
kill.head()

In [ ]:
share_race_city.head()

In [ ]:
def detect_outliers(kill,features):
    outlier_indices = []
    
    for c in features : 
        
        #1st quartile 
        Q1 = np.percentile(kill[c],25)
        
        #3rd quartile 
        Q3 = np.percentile(kill[c],75)
        
        #IQR
        
        IQR = Q3-Q1
        
        #outlier step 
        outlier_step = IQR*1.5
        
        
        #detect outlier and their indeces 
        outlier_list_col = kill[(kill[c]< Q1 - outlier_step) | (kill[c] > Q3+outlier_step)].index
        
        
        #stop indeces 
        outlier_indices.extend(outlier_list_col)
        
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list( i for i, v in outlier_indices.items() if v > 2 )
    
    return multiple_outliers

In [ ]:
kill.loc[detect_outliers(kill,["age"])]


In [ ]:
kill.isnull().sum()

In [ ]:
share_race_city.isnull().sum()

In [ ]:
kill.age.describe()

In [ ]:
def plot_hist(variable):
    plt.figure(figsize=(12,5))
    plt.hist(kill[variable], bins=50)
    plt.xlabel(variable)
    plt.ylabel("Frequence")
    plt.title("{} distribution with hist" .format(variable))
    plt.show()

In [ ]:
numericVar = ["age"]
for n in numericVar:
    plot_hist(n)

In [ ]:
# Percentage of state's population according to races that are black,white,native american, asian and hispanic
share_race_city.replace(['-'],0.0,inplace = True)
share_race_city.replace(['(X)'],0.0,inplace = True)
share_race_city.loc[:,['share_white','share_black','share_native_american','share_asian','share_hispanic']] = share_race_city.loc[:,['share_white','share_black','share_native_american','share_asian','share_hispanic']].astype(float)
area_list = list(share_race_city['Geographic area'].unique())
share_white = []
share_black = []
share_native_american = []
share_asian = []
share_hispanic = []
for i in area_list:
    x = share_race_city[share_race_city['Geographic area']==i]
    share_white.append(sum(x.share_white)/len(x))
    share_black.append(sum(x.share_black) / len(x))
    share_native_american.append(sum(x.share_native_american) / len(x))
    share_asian.append(sum(x.share_asian) / len(x))
    share_hispanic.append(sum(x.share_hispanic) / len(x))

# visualization
f,ax = plt.subplots(figsize = (16,15))
sns.barplot(x=share_white,y=area_list,color='green',alpha = 0.5,label='White' )
sns.barplot(x=share_black,y=area_list,color='blue',alpha = 0.7,label='African American')
sns.barplot(x=share_native_american,y=area_list,color='cyan',alpha = 0.6,label='Native American')
sns.barplot(x=share_asian,y=area_list,color='yellow',alpha = 0.6,label='Asian')
sns.barplot(x=share_hispanic,y=area_list,color='red',alpha = 0.6,label='Hispanic')

ax.legend(loc='upper right',frameon = True)     
ax.set(xlabel='Percentage of Races', ylabel='States',title = "Percentage of State's Population According to Races ")

In [ ]:
# Race rates according in kill data 
kill.race.dropna(inplace = True)
labels = kill.race.value_counts().index
colors = ['grey','blue','red','yellow','green','brown']
explode = [0,0,0,0,0,0]
sizes = kill.race.value_counts().values

# visual
plt.figure(figsize = (10,10))
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.2f%%')
plt.title('Killed People According to Races',color = 'blue',fontsize = 15)